In [1]:

import requests

r = requests.get('https://www.mediafax.ro')
print(r.status_code)

200


In [2]:
print(r.content)

b'<!DOCTYPE html>\n<html xmlns="http://www.w3.org/1999/xhtml" lang="ro" xml:lang="ro">\n\n    <head>\n\n        <meta name="google-site-verification" content="F84kaIgAlvwDvz__IYwu4zUHAExspfmJjzJ6ArraBh0" />\n        <meta name="google-site-verification" content="83qB9dMT3PdyVPeSXH8QpECV9DvGO-haV-FVBsQdLIo" />\n        <meta name="google-site-verification" content="llssVK3V5VYLD62XCUltG3dMTX62AlqwOWEJUGgNJmk" />\n\n        <meta http-equiv="Content-type" content="text/html;charset=UTF-8" />\n        <link rel="shortcut icon" href="/favicon.ico" />\n        <link rel="stylesheet" type="text/css" href="/style.css?v=10,11" media="screen" />\n\n         <script type="text/javascript">  </script>         <title>\xc5\x9etiri de ultima ora si ultimele \xc5\x9ftiri - \xc5\x9ftiri online - Mediafax\n</title>\n\n\t\t<meta property="fb:app_id" content="816679868905651"/>\n\t<meta property="fb:pages" content="109837159037642" />\n\t<meta property="og:site_name" content="Mediafax.ro" />\n\t<meta pro

In [3]:
import lxml.html

root = lxml.html.fromstring(r.content)
a_tags = root.xpath('//*[@id="content"]/div[3]/ul/li/p/a')
print(a_tags)

[<Element a at 0x10da3df90>, <Element a at 0x10da3df40>, <Element a at 0x10da3def0>, <Element a at 0x10da3c4b0>, <Element a at 0x10da3c460>, <Element a at 0x10da3c410>, <Element a at 0x10da3c3c0>, <Element a at 0x10da3c370>, <Element a at 0x10da3c320>, <Element a at 0x10da3c2d0>, <Element a at 0x10da2f930>, <Element a at 0x10da2fa20>, <Element a at 0x10da2f8e0>, <Element a at 0x10da2f890>, <Element a at 0x10da2f840>, <Element a at 0x10da2f610>, <Element a at 0x10da2f5c0>, <Element a at 0x10da2f570>]


In [4]:
url_list = []
for element in a_tags:
    href = element.attrib['href']
    if href.startswith('http://') or href.startswith('https://'): # if it's from a different website
        # url_list.append(href)
        pass
    elif href.startswith('/'):
        url_list.append('https://www.mediafax.ro' + href)
    else:
        url_list.append('https://www.mediafax.ro/' + href)
    
print(url_list)


['https://www.mediafax.ro/externe/cameron-oprirea-vanzarilor-de-arme-catre-israel-nu-ar-face-decat-sa-intareasca-hamas-22374819', 'https://www.mediafax.ro/externe/blinken-israelul-nu-are-un-plan-credibil-pentru-protejarea-civililor-din-rafah-22374813', 'https://www.mediafax.ro/externe/tara-din-europa-care-vrea-sa-taie-drastic-impozitul-pe-profit-pentru-a-atrage-investitorii-22374724', 'https://www.mediafax.ro/externe/israelul-a-trimis-tancuri-in-nordul-fasiei-gaza-dupa-o-noapte-cu-bombardamente-intense-22374697', 'https://www.mediafax.ro/externe/joe-biden-razboiul-s-ar-termina-maine-daca-hamas-ar-elibera-ostaticii-de-ce-refuza-organizatia-terorista-sa-i-predea-pe-cei-128-de-ostatici-din-care-nu-se-stie-cati-mai-sunt-in-viata-22374642', 'https://www.mediafax.ro/social/reactia-purtatorului-de-cuvant-al-bor-dupa-eurovision-victoria-lui-nimeni-nemo-una-zdrobitoare-asupra-firescului-22374634', 'https://www.mediafax.ro/tags/VIDEO', 'https://www.mediafax.ro/cultura-media/elvetia-a-castigat-co

In [5]:
from collections import namedtuple

Item = namedtuple('Item', ['id', 'category', 'text'])
list_items = []

for url in url_list:
    article_id = url.split('-')[-1]
    category = url.split('/')[3]
    r = requests.get(url)
    url_root = lxml.html.fromstring(r.content)
    paragraphs = url_root.xpath('//*[@id="article_text_content"]/div[1]/p')
    text = ''
    for paragraph in paragraphs:
        text += paragraph.text_content().strip() + ' '
    item = Item(article_id, category, text)
    list_items.append(item)


In [6]:
# create the raw csv file 
import csv

with open('../../data/raw/romanian_news.csv', 'a') as f:
    writer = csv.writer(f)
    writer.writerow(Item._fields)
    for item in list_items:
        writer.writerow(item)

In [7]:
import spacy
nlp = spacy.load('ro_core_news_sm')

list_clean_items = []
for item in list_items:
    doc = nlp(item.text)
    ne = [ent.text for ent in doc.ents]
    clean_text = []
    for token in doc:
        if token.text in ne:
            clean_text.append("$NE$")
        else:
            clean_text.append(token.text)
    item = Item(item.id, item.category, ' '.join(clean_text))
    list_clean_items.append(item)

print(list_clean_items)

[Item(id='22374819', category='externe', text='$NE$ $NE$ de Externe , David $NE$ , a declarat că , $NE$ nu va susţine o ofensivă terestră majoră în oraşul $NE$ din $NE$ , Marea Britanie nu va copia planurile $NE$ de a opri unele vânzări    de arme către $NE$ . " A anunţa $NE$ şi simplu $NE$ că ne vom schimba abordarea privind exporturile de arme ar face $NE$ mai puternic şi ar face mai puţin probabilă o înţelegere privind $NE$ ( $NE$ $NE$    deţinuţi de $NE$ - n.r . ) " , a arătat $NE$ .   $NE$ a spus că Marea Britanie furnizează doar $NE$ din armele $NE$ , iar pe de altă parte a avertizat că autorităţile de la Tel Aviv trebuie să facă mai mult pentru a proteja civilii şi a permite    trecerea ajutorului umanitar în $NE$ , relatează $NE$ . Săptămâna aceasta , $NE$ $NE$ , Joe Biden , a afirmat că $NE$ " nu vor furniza arme " dacă $NE$ va continua cu invazia planificată a $NE$ , unde se adăpostesc aproximativ 1,4 milioane de    $NE$ . $NE$ a declarat că va continua operaţiunile planifica

In [8]:
# create the clean csv file 
import csv

with open('../../data/clean/romanian_news.csv', 'a') as f:
    writer = csv.writer(f)
    writer.writerow(Item._fields)
    for item in list_clean_items:
        writer.writerow(item)